In [1]:
import uuid
import os
from dotenv import load_dotenv
from function import State, retrieve, evaluate_document, generate, transform_query, web_search, decide_next_step, update_history, retrieve_memory
from conversationMemory import ConservationMemory
from Retriever_memory import SemanticMemoryRetriever
from langgraph.graph import START, END , StateGraph
from format import format_response_for_display


d:\RAG\medical_rag\function.py:5: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from agent import retrieve_grader, question_rewriter
d:\RAG\medical_rag\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\RAG\medical_rag\processing_document.py:16: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprec

In [2]:
memory = ConservationMemory()
semantic_memory = SemanticMemoryRetriever()
semantic_memory.add_memory(memory.get_all_history())


In [3]:
workflow = StateGraph(State)
workflow.add_node("retriever_memory", retrieve_memory)
workflow.add_node("retriever", retrieve)
workflow.add_node("grade_document", evaluate_document)
workflow.add_node("generate", generate)
workflow.add_node("transform_query", transform_query)
workflow.add_node("web_search_tool", web_search)
workflow.add_node("update_history", update_history)

workflow.add_edge(START, "retriever_memory")
workflow.add_edge("retriever_memory", "retriever")
workflow.add_edge("retriever", "grade_document")
workflow.add_conditional_edges(
    "grade_document",
    decide_next_step,
    {
        "transform_query": "transform_query",
        "generate":"generate"
    }
)
workflow.add_edge("transform_query", "web_search_tool")
workflow.add_edge("web_search_tool", "generate")
workflow.add_edge("generate", "update_history")
workflow.add_edge("update_history", END)

app = workflow.compile()

In [4]:
#from IPython.display import Image, display
#try:
#    display(Image(app.get_graph(xray=True).draw_mermaid_png()))
#except Exception:
#    # This requires some extra dependencies and is optional
#    pass

In [6]:
def chat_with_rag():
    print("Chat with the Memory-Enhanced RAG system (type 'exit' to quit, 'clear' to clear current session)")
    chat_history = memory.get_recent_history()
    
    print(f"Loaded {len(memory.get_all_history())} previous interactions from memory")
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'exit':
            break
        elif user_input.lower() == 'clear':
            memory.clear_current_session()
            chat_history = []
            print("Current session cleared.")
            continue
        
        initial_state = {
            "question": user_input,
            "chat_history": chat_history,
            "document": [],  
            "generation": "", 
            "web_search": "no",  
            "memory_context": ""  
        }
        
        for output in app.stream(initial_state):
            for key, value in output.items():
                if key == "update_history":
                    chat_history = value.get("chat_history", [])
                    print(f"Assistant: {value.get('generation', '')}")
        print()

if __name__ == "__main__":
    chat_with_rag() 

Chat with the Memory-Enhanced RAG system (type 'exit' to quit, 'clear' to clear current session)
Loaded 8 previous interactions from memory
---RETRIEVING MEMORIES---
Retrieved 1 relevant memories
--RETRIEVE--
Query: mention the installation section
Đã tìm thấy vector database với 7 mục, đang tải...
Retrieved documents from main retriever: 5
---Check document relevant to question---
Evaluating document: IMAGE ANALYSIS:
Okay, I will analyze the image acc...
---Document relevant---
Evaluating document: *   \[12]: Warming table
    *   \[13]: Hand rest ...
---Document not relevant--- Score: no
Evaluating document: TEXT CONTENT:
I. ĐIỀU KIỆN LẮP ĐẶT
Chú ý: Không đặ...
---Document relevant---
Evaluating document: *   **Device Structure (Top Image):**
    *   \[1]...
---Document not relevant--- Score: no
Evaluating document: IMAGE ANALYSIS:
Okay, I will analyze the provided ...
---Document not relevant--- Score: no
Web search recommended: 2/5 relevant docs
---ASSESS DOCUMENT QUALITY---
--DECI